## Setup a regression experiment

In [ ]:
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

boston = load_boston()
feature_names = list(boston.feature_names)
df = pd.DataFrame(boston.data, columns=feature_names)
df["target"] = boston.target
# df = df.sample(frac=0.1, random_state=1)
train_cols = df.columns[0:-1]
label = df.columns[-1]
X = df[train_cols]
y = df[label]

seed = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

## Explore the dataset

In [ ]:
from interpret import show
from interpret.data import Marginal

marginal = Marginal().explain_data(X_train, y_train, name = 'Train Data')
show(marginal)

## Train the Explainable Boosting Machine (EBM)

In [ ]:
from interpret.glassbox import ExplainableBoostingRegressor

ebm = ExplainableBoostingRegressor(random_state=seed, n_jobs=-1)
ebm.fit(X_train, y_train)   #Works on dataframes and numpy arrays

## Global Explanations: What the model learned overall (shows the top *15* most important features)

In [ ]:
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)

## Print all feature importances

The first 15 should be the same as what is shown in the global explanation above.

In [ ]:
from interpret.visual.plot import sort_take

data_dict = sort_take(ebm_global._internal_obj["overall"], sort_fn=lambda x: -abs(x), top_n=100, reverse_results=False)
for i in range(len(data_dict["names"])):
    print("{}: {}".format(data_dict["names"][i], data_dict["scores"][i]))

## Additional Code to Compute Feature Group Importances and Append Feature Group Importances to Global Explanation

In [ ]:
import numpy as np

def get_feature_group_importance(feature_group, ebm, X, contributions=None):
    """Computes the feature importance for a group of features
 
    Args:
        feature_group: A list of feature names
        ebm: A fitted EBM
        X: Numpy array of samples
        contributions (optional): Contributions of all features per row
 
    Returns:
        A double
    """
    if contributions is None:
        _, contributions = ebm.predict_and_contrib(X)
    abs_sum_per_row = np.empty(len(contributions), np.float64)
 
    # For all rows in the dataset
    for i in range(len(contributions)):
        sum = 0.0
        # For all features in feature_group
        for j, feat_name in enumerate(ebm.get_feature_names_out()):
            if feat_name in feature_group:
                sum += contributions[i][j]
        abs_sum_per_row[i] = abs(sum)
    
    return np.average(abs_sum_per_row)
 
 
def get_group_and_individual_importances(feature_group, ebm, X):
    """Utility function to compute the feature importance for a group 
       of features as well as each feature in the group
 
    Args:
        feature_group: A list of feature names
        ebm: A fitted EBM
        X: Numpy array of samples
 
    Returns:
        A list of tuples, where each tuple is in the form (feature_name, importance)
    """
    _, contributions = ebm.predict_and_contrib(X)
    importances = []
 
    for feature in feature_group:
         importances.append( (feature, get_feature_group_importance(feature, ebm, X, contributions)) )
 
    importances.append( (feature_group, get_feature_group_importance(feature_group, ebm, X, contributions)) )
    
    return importances
 
def append_feature_importance(feature_name, feature_importance, global_exp):
    """ Appends a feature name and importance to the global explanation, which
        will only be displayed in the "Summary" Graph
 
    Args:
        feature_name (string)
        feature_importance (double)
        global_exp: An EBM Global Explanation
    """
    if global_exp._internal_obj is not None and global_exp._internal_obj["overall"] is not None:
        global_exp._internal_obj["overall"]["names"].append(feature_name)
        global_exp._internal_obj["overall"]["scores"].append(feature_importance)
    else:
        print("It was not possible to append feature {} to the global explanation.".format(feature_name))



## Compute three different feature group importances

In [ ]:
ebm_global = ebm.explain_global(name='EBM')

feature_group1 = ["RM", "NOX"]
fg1_importance = get_feature_group_importance(feature_group1, ebm, X)
print("Importance for feature group {}: {}".format(feature_group1, fg1_importance))

feature_group2 = ["CHAS", "ZN"]
fg2_importance = get_feature_group_importance(feature_group2, ebm, X)
print("Importance for feature group {}: {}".format(feature_group2, fg2_importance))

feature_group3 = ["DIS x LSTAT", "TAX x LSTAT"]
fg3_importance = get_feature_group_importance(feature_group3, ebm, X)
print("Importance for feature group {}: {}".format(feature_group3, fg3_importance))

## Append the feature group importances to the global explanation

The third feature group importance won't be shown in the graph because it's not in the top 15.

In [ ]:
append_feature_importance("RM & NOX", fg1_importance, ebm_global)
append_feature_importance("CHAS & ZN", fg2_importance, ebm_global)
append_feature_importance("DIS x LSTAT & TAX x LSTAT", fg3_importance, ebm_global)

show(ebm_global)

## Print Importance for all Features and Feature Groups

This way you see all feature and feature group importances, including "DIS x LSTAT & TAX x LSTAT" which didn't make it in the top 15.

In [ ]:
data_dict = sort_take(ebm_global._internal_obj["overall"], sort_fn=lambda x: -abs(x), top_n=100, reverse_results=False)
for i in range(len(data_dict["names"])):
    print("{}: {}".format(data_dict["names"][i], data_dict["scores"][i]))